In [1]:
import pandas as pd

In [7]:
import re

In [2]:
df = pd.read_csv('topical_chat.csv')

In [7]:
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [12]:
df.groupby('conversation_id')['message'].agg(list)

conversation_id
1       [ Are you a fan of Google or Microsoft?,  Both...
2       [ do you like dance?,  Yes  I do. Did you know...
3       [ Hey what's up do use Google very often?I rea...
4       [ Hi!  do you like to dance?,  I love to dance...
5       [ do you like dance?,  I love it. Did you know...
                              ...                        
8624    [ Good Morning!,  Good morning. How are you? S...
8625    [ Do you watch television at all? ,  I do. I l...
8626    [ Do you watch a lot of TV?,  I do! I love gam...
8627    [ Hi, how are you?,  Well thank you. Do you en...
8628    [ Are you a fan of football by any chance?,  Y...
Name: message, Length: 8628, dtype: object

In [46]:
import os
import random

import pandas as pd
import spacy
from spellchecker import SpellChecker

# Load SpaCy language model
nlp = spacy.load("en_core_web_sm")

# Create a SpellChecker object
spell = SpellChecker()

def spell_check(text):
    # Parse the text using SpaCy
    correction = []
    tokens = [((match.start(),match.end()),match.groups()[0]) for match in re.finditer(r'([a-zA-Z\']+)',text)]
    
    lastSpanEnd = 0
    for (s,e), token in tokens:
      c = spell.correction(token)
      correction.append(text[lastSpanEnd:s])
      correction.append(c if c is not None else text[s:e])
      lastSpanEnd = e
    correction.append(text[lastSpanEnd:])
    
    return ''.join(correction)


class ConversationsRepo:
  ConversationsCount = 8628
  chat_id = None
  @staticmethod
  def _get(i: int|None = None):
    """gets a chat from the Sample data

    Args:
        i (int): The number of the conversation [1, 8628]
    """
    if i is None:
      i = random.randint(1,ConversationsRepo.ConversationsCount)
    if i < 1 or i > ConversationsRepo.ConversationsCount:
      raise ValueError(i)
    ConversationsRepo.chat_id = i
    
    # script_directory = os.path.dirname(os.path.abspath(__file__))
    # file_path = os.path.join(script_directory, 'topical_chat.csv')
    file_path = './topical_chat.csv'
    
    df = pd.read_csv(file_path)
    chats = df.groupby('conversation_id')['message'].agg(list)
    chat = chats[i]
    chat = list(map(spell_check,chat))
    return chat
  @staticmethod
  def get(i: int|None = None):
    """gets a chat from the Sample data

    Args:
        i (int): The number of the conversation [1, 8628]
    """
    chat = list(map(spell_check,ConversationsRepo._get(i)))
    return chat

In [9]:
ex = """I agree. Only the lichens and cyanobateria would be able to thrive on Mars anyway."""

In [20]:
[((match.start(),match.end()),match.groups()[0]) for match in re.finditer(r'([a-zA-Z]+)',ex)]

[((0, 1), 'I'),
 ((2, 7), 'agree'),
 ((9, 13), 'Only'),
 ((14, 17), 'the'),
 ((18, 25), 'lichens'),
 ((26, 29), 'and'),
 ((30, 42), 'cyanobateria'),
 ((43, 48), 'would'),
 ((49, 51), 'be'),
 ((52, 56), 'able'),
 ((57, 59), 'to'),
 ((60, 66), 'thrive'),
 ((67, 69), 'on'),
 ((70, 74), 'Mars'),
 ((75, 81), 'anyway')]

In [26]:
import difflib

In [32]:
ex

'I agree. Only the lichens and cyanobateria would be able to thrive on Mars anyway.'

In [33]:
spell_check(ex)

'I agree. Only the lichens and cyanobacteria would be able to thrive on Mars anyway.'

In [31]:
[x for x in difflib.ndiff(ex.split(' '),spell_check(ex).split(' '))]

['  I',
 '  agree.',
 '  Only',
 '  the',
 '  lichens',
 '  and',
 '- cyanobateria',
 '+ cyanobacteria',
 '?        +\n',
 '  would',
 '  be',
 '  able',
 '  to',
 '  thrive',
 '  on',
 '  Mars',
 '  anyway.']

In [69]:
raw = ConversationsRepo._get(1541)
chat_id = ConversationsRepo.chat_id
corrected = ConversationsRepo.get(chat_id)
print(chat_id)
for x in difflib.ndiff(raw,corrected):
  print(x)

1541
   Good morning, how are you doing today?
   Fine. Reading is the complex cognitive process of decoding symbols to derive meaning.
   I really like reading. Did you know that Starship Troopers is on the reading list of 3/5i of the US Military Branches? 
   Its amazing. Now days reading is much reduced. People engaged in social media.
   Technology has made me read me more. It's more convenient to use my phone and read.
   Teenager mainly focused on social media. They don't read books and novel or newspaper.
   Yeah that is true. Did you know that is Lewis and token held "you laugh you lose" competitions when reading terrible poetry.
   There was "Moral panic" because of the spread of the reading of novels.
   It seems there's always been a fear of people gaining knowledge.
   elder's game was suggested reading for many military organizations, including the united states marine corps.
   I really liked that book. Did you know that the FDA burned six tons of literature in an act of 